Importo las librerias para poder hacer el proyecto

In [1]:
import pandas as pd
import pickle
from scipy.stats import poisson

cargo los csv y el pickle para poder usar los datos

In [2]:
tablas = pickle.load(open('grupos','rb'))
mundiales = pd.read_csv('mundiales_clean.csv')
mundial_qatar = pd.read_csv('mundialQatar_clean.csv')

ahora vamos a dividir los datos en su nivel de poder por equipo dependiendo de si es local o visitante

In [3]:
poder_local = mundiales[['local','resultado_local','resultado_visitante']]
poder_visitante = mundiales[['visitante','resultado_visitante','resultado_local']]

Renombro las columnas ya que no se va a tratar de si es o no local, sino del equipo como tal y su nivel de poder

In [4]:
poder_local.rename(columns={'local':'equipo','resultado_local':'goles_favor','resultado_visitante':'goles_contra'},inplace=True)
poder_visitante.rename(columns={'visitante':'equipo','resultado_visitante':'goles_favor','resultado_local':'goles_contra'},inplace=True)

C:\Users\Andrés Dardex\AppData\Local\Temp\ipykernel_2432\1956214220.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poder_local.rename(columns={'local':'equipo','resultado_local':'goles_favor','resultado_visitante':'goles_contra'},inplace=True)
C:\Users\Andrés Dardex\AppData\Local\Temp\ipykernel_2432\1956214220.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poder_visitante.rename(columns={'visitante':'equipo','resultado_visitante':'goles_favor','resultado_local':'goles_contra'},inplace=True)


dado que no es por localia, no sirve de nada tener separados los datos, por lo que los concateno en uno solo 

In [5]:
poder_equipo = pd.concat([poder_local,poder_visitante],ignore_index=True).groupby('equipo').mean()

Ahora vamos a "predecir" quipo a equipo quien va a ganar segun la data historica, eso depende de la cantidad de goles, por lo que no es el mejor modelo predictivo, pero luego trabajaremos en encontrar otros valores que nos ayuden a predecir mejor el ganador, pero esta es una estadistica basada en los goles historicos

In [6]:
def predecir_puntos(local, visitante):
    if local in poder_equipo.index and visitante in poder_equipo.index:
        valor_local = poder_equipo.at[local,'goles_favor'] * poder_equipo.at[visitante,'goles_contra']
        valor_visitante = poder_equipo.at[visitante,'goles_favor'] * poder_equipo.at[local,'goles_contra']
        probabilidad_local, probabilidad_visitante, probabilidad_empate = 0,0,0
        for i in range(0,10):
            for j in range(0,10):
                if i > j:
                    probabilidad_local += poisson.pmf(i,valor_local) * poisson.pmf(j,valor_visitante)
                elif i < j:
                    probabilidad_visitante += poisson.pmf(i,valor_local) * poisson.pmf(j,valor_visitante)
                else:
                    probabilidad_empate += poisson.pmf(i,valor_local) * poisson.pmf(j,valor_visitante)
        puntos_local = probabilidad_local * 3 + probabilidad_empate
        puntos_visitante = probabilidad_visitante * 3 + probabilidad_empate
        return (puntos_local,puntos_visitante)
    else:
        return(0,0) #para equipos que participan por primera vez en el mundial

In [7]:
#separo el dataframe en grupos, octavos, cuartos, semis y final para llenar datos uno por uno
mundial_qatar_grupos = mundial_qatar[:48].copy()
mundial_qatar_octavos = mundial_qatar[48:56].copy()
mundial_qatar_cuartos = mundial_qatar[56:60].copy()
mundial_qatar_semis = mundial_qatar[60:62].copy()
mundial_qatar_final = mundial_qatar[62:].copy()

ahora vamos a pedecir los puntos equipo a equipo, tomaremos cada partido del df que dividimos anteriormente, esto nos dara los puntos con los que se clasificara cada equipo.

In [8]:
for grupo in tablas:
    equipos_tablas = tablas[grupo]['Team'].values
    mundial_qatar_grupo = mundial_qatar_grupos[mundial_qatar_grupos['local'].isin(equipos_tablas)]
    for i, row in mundial_qatar_grupo.iterrows():
        local, visitante = row['local'],row['visitante']
        puntos_local, puntos_visitante = predecir_puntos(row['local'],row['visitante'])
        tablas[grupo].loc[tablas[grupo]['Team'] == local,'Pts'] += puntos_local
        tablas[grupo].loc[tablas[grupo]['Team'] == visitante,'Pts'] += puntos_visitante
    tablas[grupo] = tablas[grupo].sort_values(by=['Pts','GD','GF'],ascending=False).reset_index()
    tablas[grupo] = tablas[grupo][['Team','Pts']]
    tablas[grupo] = tablas[grupo].round(0)

ahora vamos a tomar los equipos ganadores y organizarlos como nos lo pide la segunda tabla, que es "ganador del grupo A" y "el segundo ganador del grupo B" y asi sucesivamente

In [9]:
for grupo in tablas:
    grupo_Primero = tablas[grupo].loc[0,'Team']
    grupo_Segundo = tablas[grupo].loc[1,'Team']
    mundial_qatar_octavos.replace({'Winners Group ' + grupo[-1] : grupo_Primero, 'Runners-up Group ' + grupo[-1] : grupo_Segundo},inplace=True)
mundial_qatar_octavos['Ganador'] = '-'

ahora haremos una funcion para predecir en base a su poder cual de dos equipos va a ser el ganador en cada fase.

In [10]:
def predecir_ganador(df_octavos):
    for index, row in df_octavos.iterrows():
        local, visitante = row['local'],row['visitante']
        puntos_local, puntos_visitante = predecir_puntos(local,visitante)
        if puntos_local > puntos_visitante:
            ganador = local
        else:
            ganador = visitante
        df_octavos.loc[index,'Ganador'] = ganador
    return df_octavos

In [11]:
predecir_ganador(mundial_qatar_octavos)

,local,resultado,visitante,año,Ganador
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Spain
55,Portugal,Match 56,Switzerland,2022,Portugal


ahora llenaremos el siguiente df con los ganadores, y lo haremos asi sucesivamente con cada fase.

In [12]:
def llenar_cuadros(df_octavos,df_cuartos):
    for index, row in df_octavos.iterrows():
        ganador = df_octavos.loc[index,'Ganador']
        partido = df_octavos.loc[index,'resultado']
        df_cuartos.replace({f'Winners {partido}':ganador},inplace=True)
    df_cuartos['Ganador'] = '-'
    return df_cuartos

In [13]:
llenar_cuadros(mundial_qatar_octavos,mundial_qatar_cuartos)

,local,resultado,visitante,año,Ganador
56,Germany,Match 58,Brazil,2022,-
57,Netherlands,Match 57,Argentina,2022,-
58,Spain,Match 60,Portugal,2022,-
59,England,Match 59,France,2022,-


In [14]:
predecir_ganador(mundial_qatar_cuartos)

,local,resultado,visitante,año,Ganador
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Spain,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


In [15]:
llenar_cuadros(mundial_qatar_cuartos,mundial_qatar_semis)

,local,resultado,visitante,año,Ganador
60,Netherlands,Match 61,Brazil,2022,-
61,France,Match 62,Portugal,2022,-


In [16]:
predecir_ganador(mundial_qatar_semis)

,local,resultado,visitante,año,Ganador
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


In [17]:
mundial_qatar_final.iloc[0,0] = 'Netherlands'
mundial_qatar_final.iloc[0,2] = 'Portugal'

In [18]:
mundial_qatar_final 

,local,resultado,visitante,año
62,Netherlands,Match 63,Portugal,2022
63,Winners Match 61,Match 64,Winners Match 62,2022


In [19]:
llenar_cuadros(mundial_qatar_semis,mundial_qatar_final)

,local,resultado,visitante,año,Ganador
62,Netherlands,Match 63,Portugal,2022,-
63,Brazil,Match 64,France,2022,-


In [20]:
predecir_ganador(mundial_qatar_final)

,local,resultado,visitante,año,Ganador
62,Netherlands,Match 63,Portugal,2022,Netherlands
63,Brazil,Match 64,France,2022,Brazil



### Ganador del Mundial Qatar 2022:
Brazil
### Sub-Campeon del Mundial Qatar 2022:
Francia
### Tercer Puesto del Mundial Qatar 2022:
Netherlands